In [9]:
import pandas as pd

In [10]:
df=pd.read_csv("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.csv")
df

,Dish,Hour,Requirement,Prepare,inventory available,Waste,Unfilled,used
0,Optimal Profit,74823.0,NaN,NaN,NaN,NaN,NaN,NaN
1,ice cream,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ice cream,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ice cream,2.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ice cream,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...
96,chesse ball,5.0,131.0,10.0,460.0,0.0,0.0,131.0
97,chesse ball,6.0,126.0,0.0,339.0,0.0,0.0,126.0
98,chesse ball,7.0,106.0,0.0,213.0,0.0,0.0,106.0
99,chesse ball,8.0,101.0,0.0,107.0,0.0,0.0,101.0


In [11]:
import pandas as pd

# Read the CSV file
df = pd.read_csv("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.csv")

# Save the DataFrame to an Excel file
df.to_excel("C:/Users/olw09/karthikcase/descrete-optimization/food/outputs/quantity.xlsx", index=False)


In [12]:
file_path = 'C:/Users/olw09/karthikcase/descrete-optimization/food/foodconsumption.xlsx'
df = pd.read_excel(file_path)
df=df.head(10)
df

,Dishes,Profit(₹),Loss_on_demand(₹),Loss_on_wastage₹,shelf_life(hr)
0,ice cream,20,15,12,3
1,Samosa,20,15,12,5
2,jalebi,20,15,12,7
3,aloo Paratha,20,15,12,8
4,Noodles,20,15,12,2
5,coffee,20,15,12,4
6,pastry,20,15,12,3
7,dosa,20,15,12,5
8,biryani,20,15,12,6
9,chesse ball,20,15,12,7
